In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://stackoverflow.com/questions/tagged/{}?tab=votes&page={}&pagesize=50'

In [3]:
tags = ['sql', 'javascript', 'html', 'css', 'python', 'ruby', 'java', 'go']

In [4]:
def get_data(complete):
    data = requests.get(complete)
    soup = BeautifulSoup(data.text)
    res = []
    found = soup.find_all('h3')
    for find in found:
        res.extend(find.findChildren('a', attrs={"class" : "question-hyperlink"}))
    return [a.text.strip() for a in res]

In [5]:
get_data(url.format(tags[0], 1))

['What is the difference between “INNER JOIN” and “OUTER JOIN”?',
 'How do I UPDATE from a SELECT in SQL Server?',
 'Add a column with a default value to an existing table in SQL Server',
 'How can I prevent SQL injection in PHP?',
 'How does database indexing work? [closed]',
 'How do I import an SQL file using the command line in MySQL?',
 'Finding duplicate values in a SQL table',
 'How to concatenate text from multiple rows into a single text string in SQL server?',
 'How to return only the Date from a SQL Server DateTime datatype',
 "What's the difference between INNER JOIN, LEFT JOIN, RIGHT JOIN and FULL JOIN? [duplicate]",
 'Inserting multiple rows in a single SQL query? [duplicate]',
 'Insert results of a stored procedure into a temporary table',
 'How do I perform an IF…THEN in an SQL SELECT?',
 'How to query MongoDB with “like”?',
 'Table Naming Dilemma: Singular vs. Plural Names [closed]',
 'Insert into … values ( SELECT … FROM … )',
 'Select first row in each GROUP BY group

In [6]:
X, y = [], []

for tag in tags:
    for i in range(1, 11):
        data = get_data(url.format(tag, i))
        X.extend(data)
        y.extend([tag] * len(data))
        print(f"processed {tag} on page {i}")

processed sql on page 1
processed sql on page 2
processed sql on page 3
processed sql on page 4
processed sql on page 5
processed sql on page 6
processed sql on page 7
processed sql on page 8
processed sql on page 9
processed sql on page 10
processed javascript on page 1
processed javascript on page 2
processed javascript on page 3
processed javascript on page 4
processed javascript on page 5
processed javascript on page 6
processed javascript on page 7
processed javascript on page 8
processed javascript on page 9
processed javascript on page 10
processed html on page 1
processed html on page 2
processed html on page 3
processed html on page 4
processed html on page 5
processed html on page 6
processed html on page 7
processed html on page 8
processed html on page 9
processed html on page 10
processed css on page 1
processed css on page 2
processed css on page 3
processed css on page 4
processed css on page 5
processed css on page 6
processed css on page 7
processed css on page 8
proce

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(list(zip(y, X)), columns=[tag, data])

In [9]:
df.to_csv("tags.csv", index=False)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vect = CountVectorizer(max_features=500)

In [12]:
vect.fit(X)

CountVectorizer(max_features=500)

In [13]:
len(vect.vocabulary_)

500

In [14]:
from nltk.tokenize import RegexpTokenizer

In [15]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [16]:
ps = PorterStemmer()
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))

In [17]:
def getStemmedData(data):
    data = data.lower()
    tokens = tokenizer.tokenize(data)
    new_tokens = [token for token in tokens if token not in sw]
    stemmed_token = [ps.stem(token) for token in new_tokens]
    clean_data = ' '.join(stemmed_token)
    return clean_data

def getStemmedData2(data2):
    d = []
    for data in data2:
        d.append(getStemmedData(data))
    return d

In [18]:
X = getStemmedData2(X)

In [19]:
X_mod = vect.transform(X).todense()

In [20]:
X_mod.shape

(3896, 500)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_mod, y, test_size=0.33, random_state=42)

In [23]:
from sklearn.naive_bayes import BernoulliNB

In [24]:
model = BernoulliNB()

In [25]:
model.fit(X_train, y_train)

BernoulliNB()

In [26]:
model.score(X_test, y_test)

0.6461897356143079

In [27]:
line = "How do I do JOIN in SQL?"

In [28]:
X_vec = vect.transform([line]).todense()

In [29]:
model.predict(X_vec)

array(['sql'], dtype='<U10')